<a href="https://colab.research.google.com/github/andrefs/mapi-faml-proj/blob/main/code/faml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load stuff


## Libraries

In [1]:
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

import urllib.request

## Embeddings

In [2]:
# currently not being used
def normalize(word_vec):
    norm=np.linalg.norm(word_vec)
    if norm == 0: 
       return word_vec
    return word_vec/norm

embs = {}
url = 'https://raw.githubusercontent.com/andrefs/mapi-faml-proj/main/2_clean_datasets/embeddings.txt'
response = urllib.request.urlopen(url)
lines = [l.decode('utf-8') for l in response.readlines()]
reader = csv.reader(lines, delimiter=' ')
for line in reader:
    term = line[0].replace('http://dbpedia.org/resource/' , '')
    vector = [float(x) for x in line[1:]]
    embs[term] = vector


len(embs.keys())

10706

## Relatedness pairs

In [5]:
relness = {}
url = 'https://raw.githubusercontent.com/andrefs/mapi-faml-proj/main/2_clean_datasets/relatedness_train.tsv'
response = urllib.request.urlopen(url)
lc = 0
lines = [l.decode('utf-8') for l in response.readlines()]
reader = csv.reader(lines, delimiter='\t')
for line in reader:
    lc += 1
    t1 = line[0].replace('http://dbpedia.org/resource/' , '')
    t2 = line[1].replace('http://dbpedia.org/resource/' , '')
    rel = float(line[2])
    relness[t1] = relness.get(t1,{})
    relness[t1][t2] = [float(line[2]), line[3]] # relatedness value and subset (train/test)

lc

12637

# Merge data from both sources

In [7]:
X_m = []
S_m = []
Y_m = []
for t1 in relness:
    for t2 in relness[t1]:
        Y_m.append(float(relness[t1][t2][0]))
        S_m.append(relness[t1][t2][1])
        X_m.append(np.concatenate((embs[t1],embs[t2])))
        

# Use Numpy
Y = np.array(Y_m)
X = np.matrix(X_m).astype(float)
S = np.array(S_m)

# Use pandas
X_train = pd.DataFrame(X[np.in1d(S[:], 'Train')])
X_test  = pd.DataFrame(X[np.in1d(S[:], 'Test')])
Y_train = pd.DataFrame(Y[np.in1d(S[:], 'Train')])
Y_test  = pd.DataFrame(Y[np.in1d(S[:], 'Test')])

# Insert target column into dataset
X_train['Target'] = Y_train

# 200 columns = embeddings t1
# 200 columns = embeddings t2
#   1 column  = relatedness value (target)
X_train.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,Target
count,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,...,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000,12634.000000
mean,-0.005784,-0.003849,-0.006471,0.001470,-0.000975,0.001638,0.008487,-0.007218,0.002837,0.003102,-0.003032,-0.007769,0.012584,0.008710,-0.009741,0.005933,-0.001043,0.005058,0.003125,-0.014950,0.000770,0.009375,-0.008416,-0.005823,0.000094,0.001930,-0.002556,0.008732,0.002993,0.007235,0.012268,0.003947,-0.007214,0.000301,0.006588,0.005323,-0.010910,-0.004798,-0.002721,0.005076,...,-0.001324,0.000285,0.001267,-0.014059,-0.006223,0.010473,0.003779,-0.005423,0.014272,0.008564,-0.006266,0.004603,-0.010330,-0.009634,-0.010086,-0.003992,-0.009375,0.005952,-0.007841,0.015998,-0.003050,-0.001435,0.007151,-0.007673,-0.005177,0.013480,-0.012510,-0.003363,0.004625,0.000824,-0.004589,0.003114,-0.006059,0.009906,0.005619,0.004135,0.000198,-0.000769,-0.009406,0.202508
std,0.010363,0.013415,0.010921,0.015873,0.009555,0.012014,0.022982,0.018173,0.019877,0.010617,0.015469,0.010765,0.022672,0.016232,0.014920,0.016464,0.022672,0.009641,0.012911,0.020921,0.010004,0.016180,0.014413,0.014940,0.026348,0.009950,0.015505,0.009460,0.010982,0.010180,0.015792,0.018174,0.015514,0.013240,0.010281,0.019890,0.017012,0.011142,0.012874,0.013684,...,0.014662,0.012782,0.013827,0.014967,0.012829,0.013130,0.013014,0.013417,0.016738,0.014971,0.014646,0.015006,0.012694,0.013631,0.017762,0.015086,0.021716,0.011810,0.018838,0.015908,0.008611,0.009893,0.020267,0.010484,0.012781,0.015903,0.011790,0.010382,0.008487,0.012525,0.014600,0.010991,0.012867,0.011117,0.019811,0.014581,0.009632,0.010841,0.015362,0.285275
min,-0.070691,-0.066666,-0.265590,-0.152735,-0.208192,-0.104308,-0.117777,-0.103706,-0.155246,-0.292628,-0.436979,-0.075644,-0.087675,-0.102307,-0.229997,-0.503280,-0.576806,-0.046730,-0.142459,-0.148203,-0.103551,-0.238910,-0.064874,-0.373806,-0.237818,-0.053749,-0.076468,-0.038536,-0.185282,-0.090206,-0.129383,-0.137102,-0.079424,-0.095522,-0.038815,-0.231078,-0.298210,-0.057161,-0.071691,-0.032376,...,-0.109124,-0.320076,-0.356973,-0.080544,-0.088249,-0.053687,-0.041403,-0.092535,-0.057670,-0.051905,-0.229630,-0.097756,-0.073263,-0.135089,-0.531305,-0.057748,-0.111699,-0.152144,-0.079186,-0.047341,-0.139123,-0.172307,-0.060543,-0.101662,-0.341450,-0.094072,-0.100000,-0.056204,-0.056218,-0.193806,-0.117950,-0.075355,-0.078805,-0.324372,-0.050936,-0.057190,-0.043312,-0.067530,-0.091156,0.000000
25%,-0.011079,-0.010853,-0.011474,-0.007952,-0.004814,-0.002685,-0.000812,-0.017904,-0.008969,-0.002817,-0.010634,-0.012849,-0.000063,0.001181,-0.021052,-0.001977,-0.011205,-0.000999,-0.003017,-0.032014,-0.005282,0.000108,-0.016198,-0.014959,-0.012385,-0.002841,-0.012495,0.004220,-0.001538,0.001487,0.002155,-0.004565,-0.015677,-0.005801,-0.000474,-0.005233,-0.018708,-